# connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# noise mix code

## in one directory

In [ ]:
import argparse
import array
import math
import numpy as np
import random
import wave


In [ ]:
def cal_adjusted_rms(clean_rms, snr):
    a = float(snr) / 20
    noise_rms = clean_rms / (10**a) 
    return noise_rms

def cal_amp(wf):
    buffer = wf.readframes(wf.getnframes())
    # The dtype depends on the value of pulse-code modulation. The int16 is set for 16-bit PCM.
    amptitude = (np.frombuffer(buffer, dtype="int16")).astype(np.float64)
    return amptitude

def cal_rms(amp):
    return np.sqrt(np.mean(np.square(amp), axis=-1))

def save_waveform(output_path, params, amp):
    output_file = wave.Wave_write(output_path)
    output_file.setparams(params) #nchannels, sampwidth, framerate, nframes, comptype, compname
    output_file.writeframes(array.array('h', amp.astype(np.int16)).tobytes() )
    output_file.close()


In [ ]:
if __name__ == '__main__':

        clean_file = '/content/drive/MyDrive/응급차량 프로젝트/데이터셋/fold1/police_origin.wav'
        noise_file = '/content/drive/MyDrive/응급차량 프로젝트/데이터셋/fold1/noise2_origin.wav'

        clean_wav = wave.open(clean_file, "r")
        noise_wav = wave.open(noise_file, "r")


        clean_amp = cal_amp(clean_wav)
        noise_amp = cal_amp(noise_wav)

        clean_rms = cal_rms(clean_amp)

        start = random.randint(0, len(noise_amp)-len(clean_amp))
        divided_noise_amp = noise_amp[start: start + len(clean_amp)]
        noise_rms = cal_rms(divided_noise_amp)
        
        cnt_snr = 1
        
        for i in range(-20, 21, 10):    #signal-to-noise-ratio -20, -10, 0, 10, 20 총 5개의 파일 생성
            snr = i
            adjusted_noise_rms = cal_adjusted_rms(clean_rms, snr)
            
            adjusted_noise_amp = divided_noise_amp * (adjusted_noise_rms / noise_rms) 
            mixed_amp = (clean_amp + adjusted_noise_amp)

            #Avoid clipping noise
            max_int16 = np.iinfo(np.int16).max
            min_int16 = np.iinfo(np.int16).min
            if mixed_amp.max(axis=0) > max_int16 or mixed_amp.min(axis=0) < min_int16:
                if mixed_amp.max(axis=0) >= abs(mixed_amp.min(axis=0)): 
                    reduction_rate = max_int16 / mixed_amp.max(axis=0)
                else :
                    reduction_rate = min_int16 / mixed_amp.min(axis=0)
                mixed_amp = mixed_amp * (reduction_rate)
                clean_amp = clean_amp * (reduction_rate)

            save_waveform(f'/content/drive/MyDrive/응급차량 프로젝트/데이터셋/fold1/mixed_police_2_{str(cnt_snr)}.wav', clean_wav.getparams(), mixed_amp)
            cnt_snr += 1

## in all directory

In [ ]:
'''
if __name__ == '__main__':
    data_name=[
      'police',
      'ambulance',
      'fire',
      'none'
    ]
    cnt_fold = 1
    for name in data_name:
        clean_file = f'/content/drive/MyDrive/응급차량 프로젝트/데이터셋/fold{str(cnt_fold)}/{name}.wav'
        noise_file = f'/content/drive/MyDrive/응급차량 프로젝트/데이터셋/fold{str(cnt_fold)}/noisy.wav'

        clean_wav = wave.open(clean_file, "r")
        noise_wav = wave.open(noise_file, "r")


        clean_amp = cal_amp(clean_wav)
        noise_amp = cal_amp(noise_wav)

        clean_rms = cal_rms(clean_amp)

        start = random.randint(0, len(noise_amp)-len(clean_amp))
        divided_noise_amp = noise_amp[start: start + len(clean_amp)]
        noise_rms = cal_rms(divided_noise_amp)
        
        cnt_snr = 1
        
        for i in range(-20, 21, 10):
            snr = i
            adjusted_noise_rms = cal_adjusted_rms(clean_rms, snr)
            
            adjusted_noise_amp = divided_noise_amp * (adjusted_noise_rms / noise_rms) 
            mixed_amp = (clean_amp + adjusted_noise_amp)

            #Avoid clipping noise
            max_int16 = np.iinfo(np.int16).max
            min_int16 = np.iinfo(np.int16).min
            if mixed_amp.max(axis=0) > max_int16 or mixed_amp.min(axis=0) < min_int16:
                if mixed_amp.max(axis=0) >= abs(mixed_amp.min(axis=0)): 
                    reduction_rate = max_int16 / mixed_amp.max(axis=0)
                else :
                    reduction_rate = min_int16 / mixed_amp.min(axis=0)
                mixed_amp = mixed_amp * (reduction_rate)
                clean_amp = clean_amp * (reduction_rate)

            save_waveform(f'/content/drive/MyDrive/응급차량 프로젝트/데이터셋/fold{str(cnt_fold)}/mixed_{name}_{str(cnt_snr)}.wav', clean_wav.getparams(), mixed_amp)
            cnt_snr += 1
'''

# volume

In [ ]:
def call_amp(wf):
    buffer = wf.readframes(wf.getnframes())
    # The dtype depends on the value of pulse-code modulation. The int16 is set for 16-bit PCM.
    amptitude = (np.frombuffer(buffer, dtype="int16")).astype(np.float64)
    return amptitude


def save_waveform_am(output_path, params, amp):
    output_file = wave.Wave_write(output_path)
    output_file.setparams(params) #nchannels, sampwidth, framerate, nframes, comptype, compname
    output_file.writeframes(array.array('h', amp.astype(np.int16)).tobytes() )
    output_file.close()

In [ ]:
num_of_data = 390

In [ ]:
if __name__ == '__main__':
  for constant in range(25, 101, 25):     #음량 25~100% 20%간격으로 4개씩 생성
    for noise_num in range(1, num_of_data + 1):     #소음 번호 1부터 갯수대로
      clean_file = f'/content/drive/MyDrive/응급차량 프로젝트/데이터셋/noise/noise_ch{noise_num}_origin.wav'

      clean_wav = wave.open(clean_file, "r")

      clean_amp = constant * 0.01 * call_amp(clean_wav)  
              
          #Avoid clipping noise
      max_int16 = np.iinfo(np.int16).max
      min_int16 = np.iinfo(np.int16).min
      if clean_amp.max(axis=0) > max_int16 or clean_amp.min(axis=0) < min_int16:
          if clean_amp.max(axis=0) >= abs(clean_amp.min(axis=0)): 
              reduction_rate = max_int16 / clean_amp.max(axis=0)
          else :
              reduction_rate = min_int16 / clean_amp.min(axis=0)
          clean_amp = clean_amp * (reduction_rate)

      save_waveform_am(f'/content/drive/MyDrive/응급차량 프로젝트/데이터셋/noise/noise_ch{noise_num}_{constant}%.wav', clean_wav.getparams(), clean_amp)
      #총 4 * (소음개수) 만큼 생성

# cutting

In [ ]:
def cal_adjusted_rms(clean_rms):
    noise_rms = clean_rms 
    return noise_rms

def cal_amp(wf):
    buffer = wf.readframes(wf.getnframes())
    # The dtype depends on the value of pulse-code modulation. The int16 is set for 16-bit PCM.
    amptitude = (np.frombuffer(buffer, dtype="int16")).astype(np.float64)
    return amptitude

def cal_rms(amp):
    return np.sqrt(np.mean(np.square(amp), axis=-1))

def save_waveform(output_path, params, amp):
    output_file = wave.Wave_write(output_path)
    output_file.setparams(params) #nchannels, sampwidth, framerate, nframes, comptype, compname
    output_file.writeframes(array.array('h', amp.astype(np.int16)).tobytes() )
    output_file.close()


In [ ]:
count_const = 1

In [ ]:
divide_const = 10

In [ ]:
if __name__ == '__main__':
  for num in range(10, 14):
    noise_file = f'/content/drive/MyDrive/응급차량 프로젝트/데이터셋/noise/STRAFFIC_48k/ch{num}_r8b.wav'

    noise_wav = wave.open(noise_file, "r")

    noise_amp = cal_amp(noise_wav)

    length = len(noise_amp) // divide_const

    for start in range(0, length * divide_const, length):
        divided_noise_amp = noise_amp[start: start + length]
        noise_rms = cal_rms(divided_noise_amp)

        adjusted_noise_rms = cal_adjusted_rms(noise_rms)    
        adjusted_noise_amp = divided_noise_amp * (adjusted_noise_rms / noise_rms) 
        mixed_amp = adjusted_noise_amp

        #Avoid clipping noise
        max_int16 = np.iinfo(np.int16).max
        min_int16 = np.iinfo(np.int16).min
        if mixed_amp.max(axis=0) > max_int16 or mixed_amp.min(axis=0) < min_int16:
            if mixed_amp.max(axis=0) >= abs(mixed_amp.min(axis=0)): 
                reduction_rate = max_int16 / mixed_amp.max(axis=0)
            else :
                reduction_rate = min_int16 / mixed_amp.min(axis=0)
            mixed_amp = mixed_amp * (reduction_rate)
            clean_amp = clean_amp * (reduction_rate)

        save_waveform(f'/content/drive/MyDrive/응급차량 프로젝트/데이터셋/noise/noise_ch{count_const}_origin.wav', noise_wav.getparams(), mixed_amp)
        count_const +=1